In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

Using TensorFlow backend.


In [19]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [20]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [21]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [22]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [23]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [24]:
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [25]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

In [26]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [27]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [30]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    gen = datagen.flow(x_train, y_train,
                                     batch_size=batch_size)
    model.fit_generator(gen,
                        epochs=epochs,
                        steps_per_epoch=gen.__len__(), # https://github.com/keras-team/keras/issues/10855
                        validation_data=(x_test, y_test),
                        workers=4)

Using real-time data augmentation.
Epoch 1/100
1563/1563 [==============================] - 19s 12ms/step - loss: 1.8674 - acc: 0.3164 - val_loss: 1.5902 - val_acc: 0.4292
Epoch 2/100
1563/1563 [==============================] - 18s 11ms/step - loss: 1.5875 - acc: 0.4217 - val_loss: 1.4149 - val_acc: 0.4863
Epoch 3/100
1563/1563 [==============================] - 18s 11ms/step - loss: 1.4727 - acc: 0.4660 - val_loss: 1.3131 - val_acc: 0.5250
Epoch 4/100
1563/1563 [==============================] - 19s 12ms/step - loss: 1.3950 - acc: 0.4955 - val_loss: 1.2599 - val_acc: 0.5552
Epoch 5/100
1563/1563 [==============================] - 18s 11ms/step - loss: 1.3284 - acc: 0.5256 - val_loss: 1.1673 - val_acc: 0.5819
Epoch 6/100
1563/1563 [==============================] - 18s 12ms/step - loss: 1.2756 - acc: 0.5433 - val_loss: 1.1363 - val_acc: 0.6046
Epoch 7/100
1563/1563 [==============================] - 18s 11ms/step - loss: 1.2329 - acc: 0.5597 - val_loss: 1.0751 - val_acc: 0.6243
Epoch 

1563/1563 [==============================] - 18s 11ms/step - loss: 0.7875 - acc: 0.7335 - val_loss: 0.7072 - val_acc: 0.7625
Epoch 61/100
1563/1563 [==============================] - 18s 11ms/step - loss: 0.7906 - acc: 0.7338 - val_loss: 0.7221 - val_acc: 0.7729
Epoch 62/100
1563/1563 [==============================] - 18s 11ms/step - loss: 0.7893 - acc: 0.7341 - val_loss: 0.6690 - val_acc: 0.7740
Epoch 63/100
1563/1563 [==============================] - 18s 11ms/step - loss: 0.7918 - acc: 0.7333 - val_loss: 0.7152 - val_acc: 0.7618
Epoch 64/100
1563/1563 [==============================] - 18s 11ms/step - loss: 0.7874 - acc: 0.7345 - val_loss: 0.7960 - val_acc: 0.7399
Epoch 65/100
1563/1563 [==============================] - 18s 11ms/step - loss: 0.7856 - acc: 0.7353 - val_loss: 0.7572 - val_acc: 0.7484
Epoch 66/100
1563/1563 [==============================] - 18s 11ms/step - loss: 0.7889 - acc: 0.7347 - val_loss: 0.7248 - val_acc: 0.7548
Epoch 67/100
1563/1563 [=======================

In [31]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Saved trained model at D:\godev\keras_example\saved_models\keras_cifar10_trained_model.h5 
10000/10000 [==============================] - 1s 70us/step
Test loss: 0.8472804664611816
Test accuracy: 0.7194
